In [10]:
import time
import json
import uuid
import traceback
import logging
from typing import List, Dict, Any, Optional
#from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage, BaseMessage
from langgraph.prebuilt import create_react_agent
from ibm_watsonx_ai import APIClient, Credentials
from langchain_ibm import ChatWatsonx
from models import Message, AIToolCall, Function, ChatCompletionResponse, Choice, MessageResponse
from config import OPENAI_API_KEY, WATSONX_SPACE_ID, WATSONX_API_KEY, WATSONX_URL
from token_utils import get_access_token
import os
from dotenv import load_dotenv

In [11]:
load_dotenv()
WATSONX_SPACE_ID = os.getenv('WATSONX_SPACE_ID', None)
WATSONX_API_KEY = os.getenv('WATSONX_API_KEY', None)
WATSONX_URL = os.getenv('WATSONX_URL','https://us-south.ml.cloud.ibm.com')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', None)

In [12]:
api_key = "azE6dXNyX2ZjZmZiOWIyLTA4OTctMzgxMS1hNjliLTg4YTFhMGMyMzQ3MzpmYkdWVDREWXl6OXE2NDIxT2ZpZldtOUtzNzdLT2ViSHpIRUEySWpaNVJ3PTo2akhZ"

tyvyJvMyMPDMwrQr37I5QWzxv-8GtxOQRAN_MZ6wwHny


In [17]:
import http.client
import json  # Add this import

conn = http.client.HTTPSConnection("iam.platform.saas.ibm.com")

payload = {
    "apikey": api_key
}

# Convert payload to JSON string
json_payload = json.dumps(payload)

headers = {
    'content-type': "application/json",
    'accept': "application/json"
}

conn.request("POST", "/siusermgr/api/1.0/apikeys/token", json_payload, headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"token":"eyJraWQiOiI4T3dtbDBGWEgwckkzSy1IdGNKaFBOS1FoUlBNYmVLTVRHXzdHQkZPX1RnIiwiYWxnIjoiUlMyNTYifQ.eyJpc3MiOiJodHRwczovL3NpdXNlcm1ndG1zLW1zcC11c2VyLW1hbmFnZXIuYXBwcy5hcC1kcC0wMDEua29oMy5wMS5vcGVuc2hpZnRhcHBzLmNvbS9zaXVzZXJtZ3IvYXBpLzEuMCIsImF1ZCI6ImNybjp2MTphd3M6cHVibGljOnd4bzp1cy1lYXN0LTE6c3ViLzIwMjQwNTI4LTA0NTItMjc1OS0zMDBhLWZmMDhiMWJjYTkxMToyMDI0MDYwMy0xNTA0LTI2MDQtMzAwMi01NDVkMzljZTlhODA6OiIsImV4cCI6MTczNjgxODIzOSwianRpIjoiSDJyVWs3c2c1dmNjaW1STDZ2T0dUZyIsImlhdCI6MTczNjgxMTAzOSwibmJmIjoxNzM2ODExMDA5LCJ0ZW5hbnRJZCI6IjIwMjQwNjAzLTE1MDQtMjYwNC0zMDAyLTU0NWQzOWNlOWE4MCIsInN1YnNjcmlwdGlvbklkIjoiMjAyNDA1MjgtMDQ1Mi0yNzU5LTMwMGEtZmYwOGIxYmNhOTExIiwic3ViIjoiZmNmZmI5YjItMDg5Ny0zODExLWE2OWItODhhMWEwYzIzNDczIiwiZW50aXR5VHlwZSI6IlVTRVIiLCJlbWFpbCI6ImFzaHdpbi5wb3RodWt1Y2hpQGlibS5jb20iLCJuYW1lIjoiYXNod2luLnBvdGh1a3VjaGlAaWJtLmNvbSIsImRpc3BsYXluYW1lIjoiQXNod2luLlBvdGh1a3VjaGlAaWJtLmNvbSIsImlkcCI6eyJyZWFsbU5hbWUiOiJjbG91ZElkZW50aXR5UmVhbG0iLCJpc3MiOiJodHRwczovL3dvLWlibS1wcm9kLnZlcmlmeS5pYm0uY29tL29

In [26]:
# Convert JSON response to dictionary
response_dict = json.loads(data.decode("utf-8"))

# Now response_dict is a Python dictionary
print(response_dict)  # You can access it like a normal dictionary

token = response_dict["token"]


{'token': 'eyJraWQiOiI4T3dtbDBGWEgwckkzSy1IdGNKaFBOS1FoUlBNYmVLTVRHXzdHQkZPX1RnIiwiYWxnIjoiUlMyNTYifQ.eyJpc3MiOiJodHRwczovL3NpdXNlcm1ndG1zLW1zcC11c2VyLW1hbmFnZXIuYXBwcy5hcC1kcC0wMDEua29oMy5wMS5vcGVuc2hpZnRhcHBzLmNvbS9zaXVzZXJtZ3IvYXBpLzEuMCIsImF1ZCI6ImNybjp2MTphd3M6cHVibGljOnd4bzp1cy1lYXN0LTE6c3ViLzIwMjQwNTI4LTA0NTItMjc1OS0zMDBhLWZmMDhiMWJjYTkxMToyMDI0MDYwMy0xNTA0LTI2MDQtMzAwMi01NDVkMzljZTlhODA6OiIsImV4cCI6MTczNjgxODIzOSwianRpIjoiSDJyVWs3c2c1dmNjaW1STDZ2T0dUZyIsImlhdCI6MTczNjgxMTAzOSwibmJmIjoxNzM2ODExMDA5LCJ0ZW5hbnRJZCI6IjIwMjQwNjAzLTE1MDQtMjYwNC0zMDAyLTU0NWQzOWNlOWE4MCIsInN1YnNjcmlwdGlvbklkIjoiMjAyNDA1MjgtMDQ1Mi0yNzU5LTMwMGEtZmYwOGIxYmNhOTExIiwic3ViIjoiZmNmZmI5YjItMDg5Ny0zODExLWE2OWItODhhMWEwYzIzNDczIiwiZW50aXR5VHlwZSI6IlVTRVIiLCJlbWFpbCI6ImFzaHdpbi5wb3RodWt1Y2hpQGlibS5jb20iLCJuYW1lIjoiYXNod2luLnBvdGh1a3VjaGlAaWJtLmNvbSIsImRpc3BsYXluYW1lIjoiQXNod2luLlBvdGh1a3VjaGlAaWJtLmNvbSIsImlkcCI6eyJyZWFsbU5hbWUiOiJjbG91ZElkZW50aXR5UmVhbG0iLCJpc3MiOiJodHRwczovL3dvLWlibS1wcm9kLnZlcmlmeS5pYm0uY29tL2

In [29]:
import http.client
import json

# Setup the connection - using the provided endpoint
conn = http.client.HTTPSConnection("api.dl.watson-orchestrate.ibm.com")

# Headers including authentication
headers = {
    'accept': "application/json",
    'Authorization': f'Bearer {token}'  # Replace with your actual token
}

# Make the GET request with the specific instance path
endpoint_path = "/instances/20240603-1504-2604-3002-545d39ce9a80/v1/orchestrate/agents/external-chat"
conn.request("GET", endpoint_path, headers=headers)

# Get the response
res = conn.getresponse()
data = res.read()

# Convert response to dictionary
response_dict = json.loads(data.decode("utf-8"))

# Print the results
print(response_dict)

# Close the connection
conn.close()

[{'id': '3ef94da2-0f26-449a-834a-c710ad1211e4', 'tenant_id': '20240528-0452-2759-300a-ff08b1bca911_20240603-1504-2604-3002-545d39ce9a80', 'tenant_name': 'Default WxO Tenant', 'title': 'News Today v2', 'description': 'Agent to retieve current news', 'created_on': '2025-01-13T23:17:38.986273Z', 'created_by': '6460003E3U', 'created_by_username': 'Ashwin.Pothukuchi@ibm.com', 'updated_at': '2025-01-13T23:17:38.986273Z', 'tags': None, 'name': 'News Today v2', 'type': 'external', 'nickname': 'News Today v2', 'hidden': False, 'api_url': 'https://application-6a.1qn1q8csg8r5.us-east.codeengine.appdomain.cloud/chat/completions', 'auth_scheme': 'API_KEY', 'instructions': None, 'auth_config': {}, 'chat_params': {}}, {'id': 'f3d4fa1d-1b6c-4136-b962-f41ade870a84', 'tenant_id': '20240528-0452-2759-300a-ff08b1bca911_20240603-1504-2604-3002-545d39ce9a80', 'tenant_name': 'Default WxO Tenant', 'title': 'News Today', 'description': 'Agent to retieve current news', 'created_on': '2025-01-13T18:24:47.150584Z

In [ ]:
# Setup the connection
conn = http.client.HTTPSConnection("api.dl.watson-orchestrate.ibm.com")

headers = {
    'accept': "application/json",
    'Authorization': f'Bearer {token}'
}

# Construct the endpoint path with the specific agent ID
endpoint_path = f"/instances/20240603-1504-2604-3002-545d39ce9a80/v1/orchestrate/agents/external-chat/f3d4fa1d-1b6c-4136-b962-f41ade870a84"

# Make the DELETE request
conn.request("DELETE", endpoint_path, headers=headers)

# Get the response
res = conn.getresponse()
data = res.read()

# Close the connection
conn.close()
